Task 3: Building Summary Tables
It is useful to have summary files that allow you to quickly answer questions such as the following:

How have our sales-by-day changed over the last few months?
What is our most popular item in each department?
Which owners spend the most per month in each department?
The classic way to structure data to answer these questions is in a relational database. In this task, you will build the summary text files that hold this data and populate a relational database with the data.

Input
You will process your owner records in GBQ to build the summary tables.

Output
For this task, you will build a single SQLite database via Python (in a .db file) containing three tables:

Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items[1].
Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.
Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.
You will submit your Python code that builds the database.

You are welcome to generate these tables via queries in Google Big Query, export the text files, and store them locally on your machine. Then you will need to write a Python script that creates the database, creates the tables, and fills those tables. Obviously, it’d be great to do the whole thing in Python.

[1] We can identify the number of items by looking for trans_status of ‘’ or ‘ ‘ (blank string or a single space). To be completely correct you would want to remove Returns (R) and Voids (V) as well. I’ll give you a query that helps with this.

Deliverable
The Python code that creates the summary tables. The Python code that builds the database.

# Building Summary Tables

How have our sales-by-day changed over the last few months?  
What is our most popular item in each department?  
Which owners spend the most per month in each department?

### careful
-- wrong table--- fail to query entire dataset
-- wrong queries 
-- off by 10 - 20 let it go

In [1]:
import os
#import re
#import datetime 

import sqlite3

import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import pandas_gbq
#import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

# These first two values will be different on your machine. 
service_path = "/Users/chandler/Dropbox/Teaching/"
service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
gbq_proj_id = 'umt-msba' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

In [ ]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [ ]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

# Set up a client
#client = bigquery.Client(project = "umt-msba")

In [2]:
client = bigquery.Client(project = "umt-msba")

c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Summary Table 1
Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items[1].


In [10]:
query_1 = """
    SELECT ROUND(SUM(total), 2) as sales
        , EXTRACT(DATE FROM datetime) AS date
        , EXTRACT(HOUR FROM datetime) AS hour
        , COUNT(DISTINCT CONCAT(
            CAST(EXTRACT(DATE FROM datetime) AS STRING), 
            CAST(register_no AS STRING), 
            CAST(emp_no AS STRING), 
            CAST(trans_no AS STRING)
            )) AS transactioins

        ,SUM(
            CASE
            WHEN trans_status IN ('V', 'R') THEN -1
            ELSE 1
        END
        ) AS items

        FROM `umt-msba.wedge_transactions.transArchive*`
        WHERE department NOT IN (0, 15)
        AND (trans_status IS NULL 
        OR trans_status IN ('V', 'R', '', ' '))
        AND datetime BETWEEN TIMESTAMP('2015-01-01') AND TIMESTAMP('2015-07-01')
        GROUP BY date, hour
        ORDER BY date, hour;
"""

# Execute the query
try:
    query_job = client.query(query_1)  # Start the query job
    df1 = query_job.to_dataframe()  # Convert the result to a pandas DataFrame

    # View the DataFrame
    print(df1.head())  # Show the first 5 rows of the DataFrame

except Exception as e:
    print(f"Query failed: {e}")

c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


     sales        date  hour  transactioins  items
0   441.82  2015-01-01     8             18    116
1  1501.09  2015-01-01     9             51    394
2  3715.39  2015-01-01    10             87    884
3  4166.61  2015-01-01    11            128   1100
4  4982.26  2015-01-01    12            165   1328


### Summary Table 2
Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.


need to check if there are other owner numbers that need to be excluded like 3????


In [6]:
query_2 = """
    SELECT DISTINCT card_no as owner
        , EXTRACT(YEAR FROM datetime) AS year
        , EXTRACT(MONTH FROM datetime) AS month
        , ROUND(SUM(total), 2) as sales
        , COUNT(DISTINCT CONCAT(
            CAST(EXTRACT(DATE FROM datetime) AS STRING), 
            CAST(register_no AS STRING), 
            CAST(emp_no AS STRING), 
            CAST(trans_no AS STRING)
            )) AS transactions
        ,SUM(
            CASE
            WHEN trans_status IN ('V', 'R') THEN -1
            ELSE 1
        END
        ) AS items

    FROM `umt-msba.wedge_transactions.transArchive*`
    WHERE department NOT IN (0, 15)
    AND card_no != 3
    AND (trans_status IS NULL 
    OR trans_status IN ('V', 'R', '', ' '))
    AND datetime BETWEEN TIMESTAMP('2015-01-01') AND TIMESTAMP('2015-07-01')
    GROUP BY owner, year, month
    ORDER BY owner, year, month;
"""

# Execute the query
try:
    query_job = client.query(query_2)  # Start the query job
    df2 = query_job.to_dataframe()  # Convert the result to a pandas DataFrame

    # View the DataFrame
    print(df2.head())  # Show the first 5 rows of the DataFrame

except Exception as e:
    print(f"Query failed: {e}")

c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


     owner  year  month   sales  transactions  items
0  10000.0  2015      1   23.86             1     11
1  10000.0  2015      2  123.47             3     44
2  10000.0  2015      3   72.93             3     21
3  10000.0  2015      4  123.88             4     47
4  10000.0  2015      5   67.75             1     26


### Summary Table 3
Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.


In [7]:
query_3 = """
    SELECT DISTINCT upc
    , description
    , trans.department AS `department number`
    , depts.dept_name AS `department name`
    , EXTRACT(YEAR FROM datetime) AS year
    , EXTRACT(MONTH FROM datetime) AS month
    , ROUND(SUM(total), 2) as sales
    , COUNT(DISTINCT CONCAT(
        CAST(EXTRACT(DATE FROM datetime) AS STRING), 
        CAST(register_no AS STRING), 
        CAST(emp_no AS STRING), 
        CAST(trans_no AS STRING)
        )) AS transactions
    ,SUM(
        CASE
        WHEN trans_status IN ('V', 'R') THEN -1
        ELSE 1
    END
    ) AS items

    FROM `umt-msba.wedge_transactions.transArchive*` as trans
    JOIN `umt-msba.wedge_transactions.department_lookup` as depts
    ON trans.department = depts.department
    WHERE trans.department NOT IN (0, 15)
    AND card_no != 3
    AND (trans_status IS NULL 
    OR trans_status IN ('V', 'R', '', ' '))
    AND datetime BETWEEN TIMESTAMP('2015-01-01') AND TIMESTAMP('2015-07-01')
    GROUP BY upc, description, `department number`, `department name`, year, month
    ORDER BY upc, description, `department number`, `department name`, year, month;
    """
# Execute the query
try:
    query_job = client.query(query_3)  # Start the query job
    df3 = query_job.to_dataframe()  # Convert the result to a pandas DataFrame

    # View the DataFrame
    print(df3.head())  # Show the first 5 rows of the DataFrame

except Exception as e:
    print(f"Query failed: {e}")

c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


       upc  description  department number department name  year  month  \
0  0.01DP2      PRODUCE                2.0         PRODUCE  2015      1   
1  0.01DP3  BULK Coupon                3.0            BULK  2015      2   
2  0.01DP4  REF GROCERY                4.0     REF GROCERY  2015      2   
3  0.01DP4  REF GROCERY                4.0     REF GROCERY  2015      4   
4  0.01DP4  REF GROCERY                4.0     REF GROCERY  2015      5   

   sales  transactions  items  
0   0.00             1      0  
1   0.01             1     -1  
2   0.00             1      0  
3   0.00             2      0  
4   0.00             2      0  


### Create a SQLite Database and Add Summary Tables

In [8]:
# Connect to the SQLite database (or create it)
conn = sqlite3.connect('summary_database.db')

# Create a cursor object
cursor = conn.cursor()

# Create table summary table 1
df1.to_sql('summary_table_1', conn, if_exists='replace', index=False)

# Create table summary table 2
df2.to_sql('summary_table_2', conn, if_exists='replace', index=False)

# Create table summary table 3
df3.to_sql('summary_table_3', conn, if_exists='replace', index=False)

1000

### Check to Make Sure Data is In Database

In [9]:
conn = sqlite3.connect('summary_database.db')

cursor = conn.cursor()

# Query to check the first summary table
cursor.execute("SELECT * FROM summary_table_1 LIMIT 5")
print(cursor.fetchall())

cursor.execute("SELECT * FROM summary_table_2 LIMIT 5")
print(cursor.fetchall())

cursor.execute("SELECT * FROM summary_table_3 LIMIT 5")
print(cursor.fetchall())

# Commit and close the connection when done
conn.commit()
conn.close()

[(441.82, '2015-01-01', 8, 18, 116), (1501.09, '2015-01-01', 9, 51, 394), (3715.39, '2015-01-01', 10, 87, 884), (4166.61, '2015-01-01', 11, 128, 1100), (4982.26, '2015-01-01', 12, 165, 1328)]
[(10000.0, 2015, 1, 23.86, 1, 11), (10000.0, 2015, 2, 123.47, 3, 44), (10000.0, 2015, 3, 72.93, 3, 21), (10000.0, 2015, 4, 123.88, 4, 47), (10000.0, 2015, 5, 67.75, 1, 26)]
[('0.01DP2', 'PRODUCE', 2.0, 'PRODUCE', 2015, 1, 0.0, 1, 0), ('0.01DP3', 'BULK Coupon', 3.0, 'BULK', 2015, 2, 0.01, 1, -1), ('0.01DP4', 'REF GROCERY', 4.0, 'REF GROCERY', 2015, 2, 0.0, 1, 0), ('0.01DP4', 'REF GROCERY', 4.0, 'REF GROCERY', 2015, 4, 0.0, 2, 0), ('0.01DP4', 'REF GROCERY', 4.0, 'REF GROCERY', 2015, 5, 0.0, 2, 0)]
